# 快速入门 GPT-4 Vison

从历史上看，语言模型系统仅接受**文本**作为输入。但是单一的输入形式，限制了大模型的应用落地范围。

随着技术发展，OpenAI 开发的 GPT-4 Turbo with Vision（简称 GPT-4V）允许模型接收**图像**作为输入，并回答关于它们的问题。

📢注意，目前在 Assistants API 中使用 GPT-4 时还不支持图像输入。

## 使用 GPT-4V 识别线上图像（URL）

![image_sample](https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg)

In [1]:
from openai import OpenAI

client = OpenAI(
    api_key='sk-90wao89yhvZWWfV530F1F1A1Ef964cD2Ae8a355704F209Df',         
    # 代理地址，填写商家中转站或自建OpenAI代理
    base_url='https://api.xiaoai.plus/v1' 
)

response = client.chat.completions.create(
  model="gpt-4-turbo",
  messages=[
    {
      "role": "user",
      "content": [
        {"type": "text", "text": "介绍下这幅图?"},
        {
          "type": "image_url",
          "image_url": {
            "url": "https://world-schools.com/wp-content/uploads/2023/01/IMG-Academy-photo-14.jpg",
          },
        },
      ],
    }
  ],
  max_tokens=300,
)

print(response.choices[0])

Choice(finish_reason='length', index=0, logprobs=None, message=ChatCompletionMessage(content='这幅图展示的是一个非常大且现代的健身房内部。从图片中可以看出，这个健身房配备了大量的运动设备，包括力量训练机器、杠铃、哑铃和有氧运动设备等。健身房的设计非常宽敞，两侧有上层平台，平台上也放置了一些运动器材。\n\n在健身房中间的墙壁上写着“Home of the Ascenders”，暗示这是名为“Ascenders”的运动队或俱乐部的训练基地。整体色调以蓝色和白色为主，营造出一种清爽且激励的氛围，同时墙上还有励志标语"WHERE THERE IS NO STRUGGLE, THERE IS NO STRENGTH"，意在激励训练者挑战自我，提升力量和耐力。\n\n整个空间被设计得极为现代化且功能化，适合进行各种体育训练', role='assistant', function_call=None, tool_calls=None))


In [2]:
response.choices[0].message.content

'这幅图展示的是一个非常大且现代的健身房内部。从图片中可以看出，这个健身房配备了大量的运动设备，包括力量训练机器、杠铃、哑铃和有氧运动设备等。健身房的设计非常宽敞，两侧有上层平台，平台上也放置了一些运动器材。\n\n在健身房中间的墙壁上写着“Home of the Ascenders”，暗示这是名为“Ascenders”的运动队或俱乐部的训练基地。整体色调以蓝色和白色为主，营造出一种清爽且激励的氛围，同时墙上还有励志标语"WHERE THERE IS NO STRUGGLE, THERE IS NO STRENGTH"，意在激励训练者挑战自我，提升力量和耐力。\n\n整个空间被设计得极为现代化且功能化，适合进行各种体育训练'

### 封装成一个函数 query_image_description

In [3]:
def query_image_description(url, prompt="介绍下这幅图?"):
    client = OpenAI(
        api_key='sk-90wao89yhvZWWfV530F1F1A1Ef964cD2Ae8a355704F209Df',         
        base_url='https://api.xiaoai.plus/v1' 
    )  # 初始化 OpenAI 客户端
    
    # 发送请求给 OpenAI 的聊天模型
    response = client.chat.completions.create(
        model="gpt-4-turbo",  # 指定使用的模型
        messages=[
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": prompt},
                    {"type": "image_url", "image_url": {"url": url}},
                ],
            }
        ],
        max_tokens=300,
    )
    
    # 返回模型的响应
    return response.choices[0].message.content


### 调用函数测试

![meme_0](https://p6.itc.cn/q_70/images03/20200602/0c267a0d3d814c9783659eb956969ba1.jpeg)

In [4]:
image_url = "https://p6.itc.cn/q_70/images03/20200602/0c267a0d3d814c9783659eb956969ba1.jpeg"
content = query_image_description(image_url)
print(content)

这幅图是一张幽默搞笑的图片，展示了两种不同的“狗”的版本，但实际上是对比了人们不同的工作状态。左边的狗被合成为拥有巨大肌肉的人类身躯，上面的文字是“16岁的我”，下方评论是“我刚进一片空明，务农挣点钱，人世间的大部分”，表达了年轻时充满活力和做出巨大贡献的感觉。

右边的图示是一只正常的柴犬，上方的文字是“工作后的我”，下方评论是“好累啊 明天还得继续，我不行你们先走，我先给你们垫背，我挺住你们忌讳”反映了工作一段时间后人们常感到的疲惫和压力，以及不得不持续工作的无奈。

这个搞笑对比表达了很多人对于年轻时与工


### 使用 GPT-4V 识别本地图像文件（Base64编码）


In [9]:
from openai import OpenAI
import base64
import requests
import json

client = OpenAI(
    api_key='sk-90wao89yhvZWWfV530F1F1A1Ef964cD2Ae8a355704F209Df',         
    base_url='https://api.xiaoai.plus/v1' 
)  # 初始化 OpenAI 客户端

def query_base64_image_description(image_path, prompt="解释下图里的内容？", max_tokens=1000):

    # 实现 Base64 编码
    def encode_image(path):
        with open(path, "rb") as image_file:
            return base64.b64encode(image_file.read()).decode('utf-8')

    # 获取图像的 Base64 编码字符串
    base64_image = encode_image(image_path)

    # 构造请求的 HTTP Header
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {client.api_key}"
    }

    # 构造请求的负载
    payload = {
        "model": "gpt-4-turbo",
        "messages": [
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": prompt},
                    {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{base64_image}"}}
                ]
            }
        ],
        "max_tokens": max_tokens
    }

    # 发送 HTTP 请求
    response = requests.post("https://api.xiaoai.plus/v1", headers=headers, json=payload)

    # 检查响应并提取所需的 content 字段
    if response.status_code == 200:
        response_data = response.json()
        content = response_data['choices'][0]['message']['content']
        return content
    else:
        return f"Error: {response.status_code}, {response.text}"

#### 使用 Assistants API生成的 GDP 40年对比曲线图

![gdp_data](./images/gdp_1980_2020.jpg)

In [10]:
content = query_base64_image_description("./images/gdp_1980_2020.jpg")
print(content)

ConnectTimeout: HTTPSConnectionPool(host='api.openai.com', port=443): Max retries exceeded with url: /v1/chat/completions (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x000001C26387A590>, 'Connection to api.openai.com timed out. (connect timeout=None)'))

#### 使用 GPT-4V 识别手写体笔记

![](./images/handwriting_0.jpg)

In [ ]:
content = query_base64_image_description("./images/handwriting_0.jpg")
print(content)

#### 在 Jupyter 标准输出中渲染 Markdown 格式内容

In [15]:
from IPython.display import display, Markdown

# 使用 display 和 Markdown 函数显示 Markdown 内容
display(Markdown(content))

这幅图展示的是一种食品的营养成分表。根据图中信息，每100克该食品含有以下营养成分：

- 热量：148千卡，占营养参考值（NRV%）的2%
- 蛋白质：0克，占0%
- 脂肪：0克，占0%
- 碳水化合物：8.5克，占3%
- 纤维：0克，占0%

这是一个低热量、低脂肪且低蛋白质的食品，主要提供的是碳水化合物。

![](./images/handwriting_1.jpg)

In [31]:
content = query_base64_image_description("./images/gpt-4v.jpg", "这张图片说明了什么？")
display(Markdown(content))

ConnectTimeout: HTTPSConnectionPool(host='api.openai.com', port=443): Max retries exceeded with url: /v1/chat/completions (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x000001C26418D270>, 'Connection to api.openai.com timed out. (connect timeout=None)'))

## Homework: 


### #1

使用 GPT-4V 识别带有手写体文字的本地图像文件，分享结果。

### #2

整合 `query_base64_image_description` 函数和 Markdown 格式渲染方法，使得输出结果更易阅读。

In [28]:
content = query_base64_image_description("./images/gpt-4v.jpg", "这张图片说明了什么？")
display(Markdown(content))

图片上的文字是：“拿一支笔只想写，写下你那甜蜜笑。”这是一句表达对写作或某人微笑的喜爱的诗句。